## Load in all the modules needed

In [ ]:
%pylab notebook
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset, num2date
import numpy as np
import os
import pandas as pd
import re

from cetbtools.ease2conv import Ease2Transform

# the set_trace() command is helpful for entering interactive
# debugging mode to step through lines of code in this notebook
from IPython.core.debugger import set_trace

In [ ]:
# Set pandas display options to display entire DataFrame, use None for all 
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 120)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', 10)

In [ ]:
# Define the local machine location of CETB data cubes
# This directory is expected to contain subdirectories in the following hierarchy
# that duplicates the hierarchy on the Google Shared Drive NSIDC-SD-CETB/v1/, 
# for example:
# dataDir/F13_SSMI/N/nc_cubes/cubes_<regionName>
user = 'Joan' #Mariah #MJWindows
if ('Joan' == user):
    dataDir = '/mnt/data3/cetb/nsidc0630_v1/' #jmr machine fringe 
    scriptDir = '/mnt/data3/cetb/ipynb_melt_onset/scripts'
elif ('Mariah' == user):
    dataDir = Path(Path.home(), 'nsidc0630_v1') # Mariah's PC or Mary Jo's Mac
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
elif ('MJWindows' == user):
    dataDir = Path('Z:/mj On My Mac/nsidc0630_v1') # Mary Jo's Windows machine
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
else:
    raise ValueError("unknown user= %s\n" % (user) )
    
%cd $scriptDir
dataDir, user

In [ ]:
from CETB_read_functions import read_Tb
from CETB_read_functions import read_Tb_std_dev
from CETB_read_functions import coords
from CETB_read_functions import calc_DAV
from CETB_read_functions import read_Tb_whole
from CETB_read_functions import find_cube_offset
from CETB_read_functions import grid_locations_of_subset
from CETB_read_functions import years_for
from CETB_algorithms import DAV_MOD
from CETB_algorithms import XPGR
from CETB_algorithms import end_high_DAV
from CETB_algorithms import D_DAV
from CETB_algorithms import Winter_DAV
from CETB_analysis import Tb_hist_annual
from CETB_analysis import DAV_hist_annual
from CETB_analysis import Tb_hist_monthly
from CETB_analysis import DAV_hist_monthly
from CETB_analysis import TbDAV_series_one_year
from CETB_analysis import early_melt_events
from CETB_analysis import min_max_series
from CETB_analysis import MOD_array

## Specify inputs

In [ ]:
#Specify region, satellite, sensor, channel, and image reconstruction algorithm of interest in file name
# this notebook will read in 2 CETB datasets so that channels/algorithms/sensors can be compared
region='WesternCA'  #make this the same syntax as cubefilenames and sub-directory #GLaIL
sat_GRD='F13'   #'AQUA' for AMSRE, 'F13','F14','F15'... for SSMI
sat_SIR= 'F13'
sensor_GRD='SSMI'  #'AMSRE', 'SSMI', etc.
sensor_SIR='SSMI'
channel_GRD='37V'  #'36V','36H', '18V','18H', etc. '19V','19H' and '37V','37H' for SSMI)
channel_SIR='37V'
alg_GRD='GRD'   #SIR or GRD
alg_SIR='SIR'


# set the sir to grd factor, depends on the channel
if (re.match('^[389]', channel_GRD)):
    sir_2_grd_factor = 8 # assume 3.125 km to 25 km
elif (re.match('^[12]', channel_GRD)):
    sir_2_grd_factor = 4 # assume 6.25 km to 25 km
else:
    raise ValueError("Cannot determine sir_2_grd_factor from channel %s\n" % (channel_GRD) )
    
cubeType_GRD = channel_GRD + '-' + alg_GRD
cubeType_SIR = channel_SIR + '-' + alg_SIR
  
if ('SSMI' == sensor_GRD) or ('SSMIS' == sensor_GRD):
    provider='CSU' 
    version='v1.*'
elif 'AMSRE' == sensor_GRD:
    provider='RSS'
    version='v1.3'


Years=years_for(sat_GRD)
#might want to truncate Years to subset if very slow during testing
#if we give it more years than available what do we want it to do? 
#warn me but return what it finds
    
hemName = 'N'    
    
# either change to the directory where the data is or specify it
datadir_GRD = "%s/%s_%s/%s/nc_cubes/cubes_%s" % (
    dataDir, sat_GRD, sensor_GRD, hemName, region)
datadir_SIR = "%s/%s_%s/%s/nc_cubes/cubes_%s" % (
    dataDir, sat_SIR, sensor_SIR, hemName, region)


# filepath patterns for each type of cubefile
prefix_GRD = "CETB.cubefile.%s.%s_%s-%s-%s-%s-%s" % (
    region, sat_GRD, sensor_GRD, channel_GRD, alg_GRD, provider, version)
prefix_SIR = "CETB.cubefile.%s.%s_%s-%s-%s-%s-%s" % (
    region, sat_SIR, sensor_SIR, channel_SIR, alg_SIR, provider, version)

# on MJ's machine
hemName = 'N'

#Enter a site name for titles of plots
# enter latitutde and longitude in decimal degrees

#Site='Low Relief Test, Alberta 49.73,-111.14'

#Site='Great Lakes Test Site, Lake Superior E of Duluth'
#lat_start=47.353097   
#lon_start=-89.861310

Site='NWT C57 04242006 Spring Migration'
lat_start=62.61
lon_start=-109.64

#Site='Great Slave Lake'
#lat_start=61.87167
#lon_start=-114.05

#Site='Iceland, Central Vatnajokull'
#lat_start=64.442   
#lon_start=-16.730 

#Site='Iceland, Katla Caldera Test'
#lat_start=63.64361111   
#lon_start=-19.20138889

#Site='Iceland, Vatnajokull Site 1'
#lat_start=64.64903   
#lon_start=-17.32128

#Site = 'Iceland, Vatnajokull Site 2'
#lat_start=64.25860278
#lon_start=-16.93241667

#Site='Iceland, Vatnajokull Site 3'
#lat_start=64.423333
#lon_start=-16.77305556
#Site='Iceland, Vatnajokull Site 4'
#lat_start=64.66222
#lon_start=-17.399444

#Site='Senator Beck Basin (SASP), Colorado'
#lat_start=37.9069   
#lon_start=-107.710

if Site=='Rabbit Ears':
    lat_start=40.5  #southern boundary
    lat_end=40.5  #northern boundary
    lon_start=-106.7  #western boundary
    lon_end=-106.7 #eastern boundary
elif Site=='Fraser':
    lat_start=39.85  #southern boundary
    lat_end=40.0  #northern boundary
    lon_start=-106.0  #western boundary
    lon_end=-105.9 #eastern boundary
elif Site=='North Park':
    lat_start=40.7  #southern boundary
    lat_end=40.7  #northern boundary
    lon_start=-106.15  #western boundary
    lon_end=-106.15 #eastern boundary
elif Site=='CLPX-LSA':
    lat_start=39.7  #southern boundary
    lat_end=41.0  #northern boundary
    lon_start=-106.7  #western boundary
    lon_end=-105.4 #eastern boundary
elif Site=='CUSTOM':
    lat_start=43  #southern boundary
    lat_end=43  #northern boundary
    lon_start=-110  #western boundary
    lon_end=-110 #eastern boundary
elif Site=='Senator Beck':
    # enter latitutde and longitude in decimal degrees
    lat_start=37.9069   #southern boundary
    lat_end=37.9069   #northern boundary
    lon_start=-107.726   #western boundary
    lon_end=-107.726 

In [ ]:
# check that the GRD filename prefix is really what we have on the local machine
prefix_GRD

In [ ]:
# get cube offset for finding row/col
find_cube_offset(region, cubeDir=datadir_SIR, cubeType=cubeType_SIR, verbose=False)

In [ ]:
# row and col numbers of the GRD pixels that include this region
rows_cols = grid_locations_of_subset(region, lat_start, lon_start, cubeDir=datadir_GRD)
rows_cols_GRD=[rows_cols[0][2],rows_cols[0][2]+1, rows_cols[1][2], rows_cols[1][2]+1]
rows_cols_GRD

In [ ]:
# row and col numbers of the envelope of 64 SIR pixels
#MJB: I am not convinced that this is a good approach to calculate these coordinates
#I need to look at this a little more closely
rows_cols_env=[rows_cols_GRD[0]*sir_2_grd_factor,rows_cols_GRD[1]*sir_2_grd_factor, rows_cols_GRD[2]*sir_2_grd_factor, rows_cols_GRD[3]*sir_2_grd_factor]
rows_cols_env

In [ ]:
# check that the directory, filename pattern and Years are what we expect
# subset Years to a shorter set if only testing, because this takes a long time
Years = Years[0:2]
datadir_SIR, prefix_SIR, Years

In [ ]:
#new cell to read whole dataset
#takes a long time, for testing substitute a year for Years
data_SIR = read_Tb_whole(datadir_SIR, prefix_SIR, Years)

#FIXME: instead of copying these elements into new variables, just
#use the values directly in subsequent cells.  This is a more direct way to
#work with the output from the read_Tb_whole function.  It also would
#make it clear in subsequent cells that, for example "cal_month" is the
#month output from the SIR call.
CETB_SIR = data_SIR['TB']   # 3-D Tb time-series array
cal_date = data_SIR['cal_date']    # 1-D array of dates
cal_year = data_SIR['cal_year']    # 1-D array of years
cal_month = data_SIR['cal_month']   # 1-D array of months

#FIXME: same issue here, instead of copying the GRD TB data, subsequent
#cells should just use data_GRD['TB']
data_GRD = read_Tb_whole(datadir_GRD, prefix_GRD, Years)
CETB_GRD = data_GRD['TB']

In [ ]:
#region version, may not need
# 'data' will be a tuple with the first element as the Tb time-series data, days, months, and years 
# to be used in plot/algorithm functions later
#data_SIR=read_Tb(datadir_SIR, prefix_SIR, Years,rows_cols_env[0],rows_cols_env[1],rows_cols_env[2],rows_cols_env[3])
#CETB_SIR=data_SIR[0]   # 3-D Tb time-series array
#cal_date=data_SIR[1]    # 1-D array of dates
#cal_year=data_SIR[2]    # 1-D array of years
#cal_month=data_SIR[3]   # 1-D array of months
#data_GRD=read_Tb(datadir_GRD, prefix_GRD, Years,rows_cols_GRD[0],rows_cols_GRD[1],rows_cols_GRD[2],rows_cols_GRD[3])
#CETB_GRD=data_GRD[0]

In [ ]:
# calculate DAV for the Tb data that was imported
DAV_SIR = calc_DAV(CETB_SIR)
DAV_GRD = calc_DAV(CETB_GRD)

In [ ]:
# calculate melt onset date, each row returned is the MOD for that year, 
# shows '3' with the first pixel in the subset to melt
# can tweak thresholds and count/window, legacy algorithm used 
# 3 occurrences in 5 day window
#DAV_threshold = 18
#Tb_threshold = 252
occurrences = 3
window = 10
DAV_threshold = 18
Tb_threshold = 252

In [ ]:
#MOD_SIR1, melt_flag_df = DAV_MOD(DAV_threshold, Tb_threshold, occurrences, window, 
#                                 DAV_SIR, CETB_SIR, Years, cal_year, cal_date, 
#                                 rows_cols_env)
#MOD_SIR1

## MAPS

In [ ]:
# Get lat/lon of each pixel in the subset of 64 in the data_SIR dict
lat_lon = np.dstack((data_SIR['latitude'], data_SIR['longitude']))
coordinates = lat_lon[rows_cols_env[0]:rows_cols_env[1],
                      rows_cols_env[2]:rows_cols_env[3]]
coordinates = coordinates.reshape(
    ((rows_cols_env[1]-rows_cols_env[0])*(rows_cols_env[3]-rows_cols_env[2])),
    2)

pixel_lats = coordinates[:,0]
pixel_lons = coordinates[:,1]
pixel_lats, pixel_lons

In [ ]:
# get array of MODs for each pixel in the 64 set to use for map
window = 10
count = 3
DAV_threshold = 18
Tb_threshold = 252
MOD_DOY_array = MOD_array(datadir_SIR, prefix_SIR, CETB_SIR, 
    DAV_SIR, rows_cols_env, cal_date, Years, window, count, 
    DAV_threshold, Tb_threshold)
MOD_DOY_array

In [ ]:
# load SIR Tb standard deviation data from netCDF file with function
Tb_std_dev=read_Tb_std_dev(datadir_SIR, prefix_SIR, Years,rows_cols_env[0],rows_cols_env[1],rows_cols_env[2],rows_cols_env[3])
#Tb_std_dev

In [ ]:
#need to run this cell for the max DAV to work
y_dims_list=list(range(len(CETB_SIR[0,:,0])))  # creates a list of the y-dimension pixel indices, used for plotting
x_dims_list=list(range(len(CETB_SIR[0,0,:])))	# creates a list of the x-dimension pixel indices, for plotting
y_s=list(range(rows_cols_env[0],rows_cols_env[1]))  # makes a list of the y(row) numbers so the dataframe of early melt events can be indexed
x_s=list(range(rows_cols_env[2],rows_cols_env[3]))  # makes a list of the x(col) numbers so the dataframe of early melt events can be indexed

# create dataframe of the 64 SIR pixels
Tb_std_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=Tb_std_dev[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        Tb_std_frame=pd.concat([Tb_std_frame,column],axis=1)
Tb_std_frame.index=cal_date
Tb_std_frame

In [ ]:
# load dataframe of Tb GRD for plotting Tb on specified day and observation
Tb_std_dev_GRD=read_Tb_std_dev(datadir_GRD, prefix_GRD, Years,rows_cols_GRD[0],rows_cols_GRD[1],rows_cols_GRD[2],rows_cols_GRD[3])
Tb_std_dev_GRD

In [ ]:
# slice of dataframe described above
Tb_sd_frame_GRD=pd.DataFrame(Tb_std_dev_GRD[:,0,0], index=cal_date)
Tb_sd_frame_GRD

In [ ]:
# load dataframe of Tb GRD for plotting Tb on specified day and observation
Tb_frame_GRD=pd.DataFrame(CETB_GRD[:,0,0], index=cal_date)
Tb_frame_GRD

In [ ]:
# load dataframe of Tb SIR for plotting Tb on specified day and observation
Tb_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=CETB_SIR[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        Tb_frame=pd.concat([Tb_frame,column],axis=1)
Tb_frame.index=cal_date
Tb_frame

In [ ]:
# get the min and max Tb of the 64 for each time step
Tb_64_min=Tb_frame.min(axis=1)
Tb_64_max=Tb_frame.max(axis=1)

In [ ]:
# compute std dev of the 64 SIR pixels
Tb_std_64=Tb_frame.std(axis=1)
Tb_std_64

In [ ]:
# take difference of the GRD_std_dev variable and the computed sd of the 64 SIR pixels
Tb_sd_diff=Tb_sd_frame_GRD[0]-Tb_std_64
Tb_sd_diff

In [ ]:
DAV_GRD_frame=pd.DataFrame(DAV_GRD[:,0,0], index=cal_date)
DAV_GRD_frame

In [ ]:
DAV_SIR_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=DAV_SIR[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        DAV_SIR_frame=pd.concat([DAV_SIR_frame,column],axis=1)
DAV_SIR_frame=DAV_SIR_frame.set_index(cal_date)
DAV_SIR_frame

In [ ]:
#Generate a time series of specified length (any window in dataset)
#Make sure to update filename in last line to export new file
fig,(ax1,ax2,ax3)=plt.subplots(3,1, sharex=True, figsize=(10,10))
#ax.xlim('2003-1-1', '2011-12-31')
ax1.set_title(str(lat_start)+'_'+str(lon_start)+' '+Site)
#Tb_sd_frame_GRD.plot(xlim=['2019-01-01','2019-12-31'])

ax1.plot(Tb_64_max, color='red', label='SIR-Tb-max')
ax1.plot(Tb_64_min, color='blue', label='SIR-Tb-min')
ax1.plot(Tb_nearest, color = 'black',label='SIR-Site')
ax1.plot(Tb_frame_GRD, color='#636363', label='GRD-Tb')
ax1.set_ylabel('Tb (K)', fontsize=16)
ax1.set_ylim(150,300)
#ax1.yticks(fontsize=12)
ax1.legend(fontsize=12)
ax2.plot(Tb_sd_frame_GRD, color='black', label='GRD-StDev')
ax2.plot(Tb_std_frame[str(rows_cols[0][0])+','+str(rows_cols[1][0])], color='#fdbb84', label='SIR-1px')
ax2.set_ylabel('Tb-STD (K)',fontsize=16)
ax2.plot(Tb_std_64, color='#4292c6',label='SIR-64px-StDev')
ax2.plot(Tb_sd_diff, label='StDevDiff_GRD-64SIR', color='green')
ax2.legend(fontsize=10,loc='lower right')
ax2.set_ylim(-12,30)
ax3.plot(DAV_GRD_frame, color='black', label='GRD-DAV')
ax3.plot(DAV_SIR_frame[str(rows_cols[0][0])+','+str(rows_cols[1][0])], color='#4292c6', label='SIR-DAV')
ax3.set_ylabel('DAV (K)',fontsize=16)
ax3.legend(fontsize=12)
ax3.set_ylim(0,100)
ax1.set_xlim('2003-01-01', '2011-12-31')

fig.autofmt_xdate()
plt.savefig('../jmr_plots/Caribou.png')


In [ ]:
# histogram of the difference between GRD_Tb_std variable and the computed std of the 64 SIR pixels
#plt.close()
fig,ax=plt.subplots()
values=Tb_sd_diff.dropna().values
ax.hist(values,bins=50, label='STD_Dif GRD-64SIR')
ax.set_title(Site+' all data')
ax.axvline(x=Tb_sd_diff.mean(), color='red', linestyle='dashed', label=Tb_sd_diff.mean().round(3))
ax.legend(fontsize=12)
ax.set_ylabel('Frequency',fontsize=14)
ax.set_xlabel('Standard Deviation Difference (K)',fontsize=14)
plt.savefig('../jmr_plots/Iceland_Katla_histdif_F18_all.png')

In [ ]:
# Plot Tb map
date='2019-05-08' # specify date of interest
obs=0 # 0 for morning or 1 for evening observation

# load Tb_std_dev array
array=Tb_frame.loc[date].values
array=array[obs]
GRD=Tb_frame_GRD.loc[date].values
GRD=GRD[obs]

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(pixel_lons, pixel_lats)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=array, s=100, marker='s',lw=0, cmap='Greys', alpha=.8)  # cmap='Blues'
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title('Tb_'+date+' '+str(obs))

# plot a red line on colorbar for the MOD for the GRD pixel
cbar=plt.colorbar()
cbar.ax.hlines(cbar.norm(GRD), 0, 1, color='red') 
plt.clim(230,250)  #color bar limits

plt.show()
#plt.savefig('../jmr_plots/PnOWL_amsre_TB_040107-1.png')
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#x,y=m(pixel_lons, pixel_lats)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
#m.scatter(x,y,c=MOD_DOY_array, s=150, marker='s',lw=0,cmap='Blues', alpha=.0)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
#plt.title('Tb - std dev')
#plt.colorbar()
plt.show()
plt.savefig('../jmr_plots/Iceland_Katla_f18_05082019AM.png')


In [ ]:
GRD


In [ ]:
# returns the average Jan-Feb DAV for each pixel for each year - SIR
Jan_Feb_DAV=Winter_DAV(CETB_SIR, cal_date, cal_year, Years, rows_cols_env)
Jan_Feb_DAV

In [ ]:
Jan_Feb_DAV_avg=Jan_Feb_DAV.mean(axis=0)
Jan_Feb_DAV_avg=Jan_Feb_DAV_avg.values
#Jan_Feb_DAV_avg=np.round(Jan_Feb_DAV_avg)
Jan_Feb_DAV_avg

In [ ]:
# returns the average Jan-Feb DAV for the GRD pixel for each year
Jan_Feb_DAV_GRD=Winter_DAV(CETB_GRD, cal_date, cal_year, Years, rows_cols_GRD)
Jan_Feb_DAV_GRD

In [ ]:
# avg Jan-Feb DAV for all years - GRD pixel
Jan_Feb_DAV_GRD_all=np.nanmean(Jan_Feb_DAV_GRD)
Jan_Feb_DAV_GRD_all

In [ ]:
# PLOT WINTER DAV MAP
# if getting errors, likely because there are 0 or only 1 early melt event for the subset
fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(pixel_lons, pixel_lats)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=Jan_Feb_DAV_avg, s=100, marker='s',lw=0,cmap='Blues', alpha=.8)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title(sensor_SIR+'-'+Site+' - DAV - (Avg '+str(Years[0])+'-'+str(Years[-1])+')')

# plot a red line on colorbar for the GRD mean - all years
cbar=plt.colorbar()
cbar.ax.hlines(cbar.norm(Jan_Feb_DAV_GRD_all), 0, 1, color='green') 

plt.show()

fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#x,y=m(pixel_lons, pixel_lats)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
#m.scatter(x,y,c=MOD_DOY_array, s=150, marker='s',lw=0,cmap='Blues', alpha=.0)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title(sensor_SIR+'-'+Site+' - DAV - (Avg '+str(Years[0])+'-'+str(Years[-1])+')')
#plt.colorbar()
plt.show()



In [ ]:
# winter DAV histogram
plt.close()
fig,ax=plt.subplots()
ax.hist(Jan_Feb_DAV_avg)
avg=np.mean(Jan_Feb_DAV_avg)  # calculate average winter DAV all pixels
ax.axvline(x=avg, color='blue', linestyle='dashed')  # plot a line for the average winter DAV for all pixels
ax.axvline(x=Jan_Feb_DAV_GRD_all, color='red')  # plot a line for winter DAV for GRD pixel
ax.set_xlabel('DAV (K)')
ax.set_ylabel('Count')
ax.set_title('Winter (Jan-Feb) DAV')

In [ ]:
#need to run this cell for the max DAV to work
y_dims_list=list(range(len(CETB_SIR[0,:,0])))  # creates a list of the y-dimension pixel indices, used for plotting
x_dims_list=list(range(len(CETB_SIR[0,0,:])))	# creates a list of the x-dimension pixel indices, for plotting
y_s=list(range(rows_cols_env[0],rows_cols_env[1]))  # makes a list of the y(row) numbers so the dataframe of early melt events can be indexed
x_s=list(range(rows_cols_env[2],rows_cols_env[3]))  # makes a list of the x(col) numbers so the dataframe of early melt events can be indexed

In [ ]:
# create array of the max DAV for each pixel for all years
DAV_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=DAV_SIR[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        DAV_frame=pd.concat([DAV_frame,column],axis=1)
DAV_frame=DAV_frame.set_index(cal_date)
#DAV_frame=DAV_frame.groupby(pd.Grouper(freq='A')).max()
DAV_max_array=DAV_frame.max().values
DAV_max_array

In [ ]:
# create array of the max DAV for each pixel for year of interest
year=2019  #specify year
DAV_max_array_year=DAV_frame[str(year)].max().values
DAV_max_array_year

In [ ]:
DAV_max_GRD=np.nanmax(DAV_GRD)
DAV_max_GRD

In [ ]:
#Note this year needs to be the same as the max dav plot below
#year=2005
DAV_max_GRD_year=np.nanmax(DAV_GRD[cal_year==year])
DAV_max_GRD_year

In [ ]:
# PLOT MAX DAV MAP
#plot_type='all'
year = 2019
plot_type=year
#NOTE YEAR SPECIFIED UP A FEW LEVELS (~3)
#year=2005

if plot_type=='all':
    array=DAV_max_array
    title=sensor_SIR+'-'+Site+' - Max DAV (K) - (Avg of '+str(Years[0])+'-'+str(Years[-1])+')'
    GRD=DAV_max_GRD
elif plot_type==year:

    array=DAV_max_array_year
    GRD=DAV_max_GRD_year
    title=sensor_SIR+'-'+Site+' - Max DAV (K) - ('+str(year)+')'

fig = plt.figure(figsize=(8,8))
# create figure and axes instances
#fig = plt.figure()
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#ax = fig.add_subplot(121)
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

x,y=m(pixel_lons, pixel_lats)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
m.scatter(x,y,c=array, s=100, marker='s',lw=0,cmap='Blues', alpha=.8)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title(title)

# plot a red line on colorbar for the GRD max
cbar=plt.colorbar()
#cbar.ax.hlines(cbar.norm(DAV_max_GRD), 0, 1, color='red') 
cbar.ax.hlines(cbar.norm(DAV_max_GRD_year), 0, 1, color='red')
plt.show()

fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])

# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_start,lat_0=lat_start,lat_ts=90.,\
            llcrnrlat=(lat_start-.7),urcrnrlat=(lat_start+.7),\
            llcrnrlon=(lon_start-.7),urcrnrlon=(lon_start+.7),\
            rsphere=6371200.,resolution='l',area_thresh=10000, epsg=3857)
m.drawcoastlines()
m.drawcountries()
m.drawstates()
#m.etopo(scale=5, alpha=0.7)
#m.bluemarble(scale=1)
parallels = np.arange(0.,90,0.5)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,0.5)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
m.arcgisimage(service='World_Physical_Map', xpixels = 1500, verbose= True)

#x,y=m(pixel_lons, pixel_lats)
#m.plot(x,y, 'bs', markersize=11, markeredgewidth=1, alpha=0.2)
#m.scatter(x,y,c=MOD_DOY_array, s=150, marker='s',lw=0,cmap='Blues', alpha=.0)
#x,y=m(ISAlons,ISAlats)
#m.plot(x,y,'ro',markersize=2,markeredgewidth=0.0)
plt.title(title)
#plt.colorbar()
plt.show()

## Histograms

In [ ]:
# generate histogram - SIR - pick year 
year=2019  # specify year here
Tb_threshold=252  #specify threshold
data = CETB_SIR[cal_year==year] # CETB_data for all pixels in subset
data = data[data>=0]
bins = range(150,300)  # bins for histogram
fig,ax=plt.subplots()
ax.axvline(x=Tb_threshold, color='red')
ax.hist(data, bins)
ax.set_title(str(year)+' SIR Histogram')
#ax.set_title(prefix+'row'+str(x)+'col'+str(y))
ax.set_xlabel('Brightness Temp (K)')
#fig.savefig('/home/mij216/Documents/GLaIL/'+prefix+'row'+str(x)+'col'+str(y)+'.png')
plt.savefig('../jmr_plots/Iceland_Katla_F18_histsir2019.png')

In [ ]:
# generate histogram - SIR - all data
#year=2005
#Tb_threshold=252
data = CETB_SIR[:] # CETB_data for all pixels in subset
data = data[data>=0]
bins = range(150,300)  # bins for histogram
fig,ax=plt.subplots()
ax.hist(data, bins)
ax.set_title('All data SIR Histogram')
ax.axvline(x=Tb_threshold, color='red')
#ax.set_title(prefix+'row'+str(x)+'col'+str(y))
ax.set_xlabel('Brightness Temp (K)')
#fig.savefig('/home/mij216/Documents/GLaIL/'+prefix+'row'+str(x)+'col'+str(y)+'.png')
plt.savefig('../jmr_plots/Iceland_Katla_F182019_HISTsirall.png')

In [ ]:
# generate histogram - GRD - all data
data = CETB_GRD[:] # CETB_data for all pixels in subset
data = data[data>=0]
bins = range(150,300)  # bins for histogram
fig,ax=plt.subplots()
ax.hist(data, bins)
ax.set_title('All data GRD Histogram')
ax.axvline(x=Tb_threshold, color='red')
#ax.set_title(prefix+'row'+str(x)+'col'+str(y))
ax.set_xlabel('Brightness Temp (K)')
#fig.savefig('/home/mij216/Documents/GLaIL/'+prefix+'row'+str(x)+'col'+str(y)+'.png')
plt.savefig('../jmr_plots/Iceland_Katla_f18_histGRDall.png')

In [ ]:
# min/max/avg time series and histogram of the 64 SIR pixels, plus the GRD pixel
year=2019
min_max_series(CETB_GRD, CETB_SIR, cal_date, cal_year, year, str(lat_start)+'_'+str(lon_start))
plt.savefig('../jmr_plots/Iceland_site4Vatna_f18_minmaxhists2019.png')

## TIME SERIES

In [ ]:
# Tb and DAV time series for GRD pixel, pick one year
#year=2003
Tb_threshold=252
DAV_threshold=18
TbDAV_series_one_year(CETB_GRD,DAV_GRD, cal_date, cal_year, year, Tb_threshold, DAV_threshold)

In [ ]:
# Tb and DAV time series for SIR pixels, pick one year
#year=2003
#Tb_threshold=252
#DAV_threshold=18
TbDAV_series_one_year(CETB_SIR, DAV_SIR, cal_date, cal_year, year, Tb_threshold, DAV_threshold)

In [ ]:
# Tb and DAV time series of the single SIR pixel closest to the user-specified lat/lon at top of notebook
year=2019
#Tb_threshold=252
#DAV_threshold=18
data=read_Tb(datadir_SIR, prefix_SIR, Years,rows_cols[0][0],rows_cols[0][0]+1,rows_cols[1][0],rows_cols[1][0]+1)
SIR_1px=data[0]
DAV_1px=calc_DAV(SIR_1px)
TbDAV_series_one_year(SIR_1px, DAV_1px, cal_date, cal_year, year, Tb_threshold, DAV_threshold) 

## CREATE CSVs

In [ ]:
TB_SIR_1px=pd.DataFrame([SIR_1px.flatten()]).T

TB_SIR_1px.index=cal_date
TB_SIR_1px

In [ ]:
DAV_SIR_1px=pd.DataFrame([DAV_1px.flatten()]).T
DAV_SIR_1px.index=cal_date
DAV_SIR_1px

In [ ]:
# CREATE CSV of GRD and SIR TB, GRD and SIR64 ST, and GRD And SIR DAV

# CREATE CSV OF TB, DAV, SD and sirSD FOR THE GRD PIXEL - ALL YEARS
Tb_frame=pd.DataFrame(CETB_GRD[:,0,0], index=cal_date)
DAV_frame=pd.DataFrame(DAV_GRD[:,0,0], index=cal_date)
GRD_data_out=pd.concat([Tb_frame,DAV_frame,TB_SIR_1px,DAV_SIR_1px,Tb_sd_frame_GRD,Tb_std_64,Tb_sd_diff],axis=1)
GRD_data_out.columns=['GRDTb36V','GRDDAV36V','SIRTB36V','SIRDAV36V','GRD_SD_36V','SIR64_to_SD_36V','SD_diff[GRD-SIR]']
GRD_data_out

In [ ]:
GRD_data_out.to_csv('/home/jmr204/noisy/sd/'+sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'+str(lat_start)+'_'+str(lon_start)+'DATA.csv')  # modify destination to save csv

In [ ]:
# CREATE CSV OF TB FOR THE GRD PIXEL - ALL YEARS
Tb_frame=pd.DataFrame(CETB_GRD[:,0,0], index=cal_date)
Tb_frame.to_csv('/home/jmr204/noisy/noisytb_csv/'+sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'+str(lat_start)+'_'+str(lon_start)+'.csv')  # modify destination to save csv

In [ ]:
# CREATE CSV of DAV FOR THE GRD PIXEL - ALL YEARS
DAV_frame=pd.DataFrame(DAV_GRD[:,0,0], index=cal_date)
DAV_frame.to_csv('/home/jmr204/noisy/noisydav_csv/'+sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'+str(lat_start)+'_'+str(lon_start)+'.csv') # modify destination to save csv

In [ ]:
# CREATE CSV of TB FOR 64pixel subset OF SIR PIXELS - ALL YEARS
Tb_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=CETB_SIR[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        Tb_frame=pd.concat([Tb_frame,column],axis=1)
Tb_frame=Tb_frame.set_index(cal_date)
Tb_frame.to_csv('/home/jmr204/noisy/noisytbSIR_csv/'+sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'+str(lat_start)+'_'+str(lon_start)+'.csv')  # modify destination to save csv

In [ ]:
# CREATE CSV of DAV FOR 64pixel subset OF SIR PIXELS - ALL YEARS
DAV_frame=pd.DataFrame()
for i in y_dims_list:
    for j in x_dims_list:
        column=pd.DataFrame(data=DAV_SIR[:,i,j], columns=[str(y_s[i])+','+str(x_s[j])])
        DAV_frame=pd.concat([DAV_frame,column],axis=1)
DAV_frame=DAV_frame.set_index(cal_date)
DAV_frame.to_csv('/home/jmr204/noisy/noisydavSIR_csv/'+sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'_'+str(lat_start)+'_'+str(lon_start)+'.csv')  # modify destination to save csv

